In [5]:
import pandas as pd

# 로컬 JSON 파일 읽기
df = pd.read_csv("./data/summary_two.csv")
df.tail()


,docid,content,summary
4267,ae28101b-a42e-45b7-b24b-4ea0f1fb2d50,비뇨기계와 순환계는 혈액이 신장을 통과하면서 폐기물과 물이 제거될 때 관여하는 두 ...,"비뇨기계는 신장과 요관을 통해 혈액 정화와 노폐물 제거에 기여하며, 순환계는 심장과..."
4268,eb727a4f-29c7-4d0c-b364-0e67de1776e9,로봇은 현대 산업에서 많은 역할을 수행할 수 있습니다. 그러나 로봇 사용의 중대한 ...,"로봇은 산업 현장에서 인간의 위험 작업을 수행하며, 조립 과정의 정확성과 일관성 유..."
4269,0c8c0086-c377-4201-81fa-25159e5435a7,"월경은 여성의 생리주기에 따라 발생하는 현상으로, 에스트로겐과 프로게스테론 수치의 ...","월경은 여성의 생리주기와 에스트로겐·프로게스테론 변화에 의해 발생하며, 자궁 내막 ..."
4270,06da6a19-ec78-404e-9640-9fc33f63c6a2,식물이 내뿜는 가스는 산소입니다. 식물은 광합성 과정을 통해 태양 에너지를 이용하여...,"식물은 광합성 과정을 통해 태양 에너지와 이산화탄소를 이용해 산소를 생성하며, 이 ..."
4271,03c36d5e-c711-4dc2-b4db-aaeb94d86395,"버퍼 오버런은 개발자들이 만든 널리 퍼져 있는 앱의 코딩 오류로써, 공격자가 시스템...","버퍼 오버런은 개발자 코딩 오류로 발생하는 보안 취약점으로, 메모리 버퍼 초과 접근..."


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4272 entries, 0 to 4271
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   docid    4272 non-null   object
 1   content  4272 non-null   object
 2   summary  4272 non-null   object
dtypes: object(3)
memory usage: 100.3+ KB


In [7]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore

class ScienceRAG:
    def __init__(self):
        self.embeddings = HuggingFaceEmbeddings(
                                    model_name="BAAI/bge-m3",
                                    model_kwargs={"device": "cuda"} ,
                                    encode_kwargs={"normalize_embeddings": True})
        self.vectorstore = FAISS.from_texts(["초기화용 더미 텍스트"], self.embeddings)
        """  # 임베딩 차원 계산
        dimension = len(self.embeddings.embed_query("test"))
    
        # 코사인 유사도를 위한 IndexFlatIP 생성
        index = faiss.IndexFlatIP(dimension)
    
        # 빈 FAISS 벡터 스토어 생성
        self.vectorstore = FAISS(
            embedding_function=self.embeddings,
            index=index,
            docstore=InMemoryDocstore(),
            index_to_docstore_id={},
            normalize_L2=True  # 벡터 정규화로 코사인 유사도 구현
        ) """

        # 요약 체인
        #self.summary_chain = science_summary_chain
        #self.summary_chain = two_step_chain
    
    def add_documents(self, df):
        """문서를 요약하여 벡터DB에 저장"""
        
        texts = df['summary'].tolist()
        metadatas = [
            {
                'docid': row['docid'],
                'content': row['content']
            } 
            for _, row in df.iterrows()
        ]
        ids = df['docid'].tolist()


        # 2. 검색용 요약을 벡터DB에 저장
        self.vectorstore.add_texts(
            texts,
            metadatas=metadatas,
            ids = ids
        )
        
       
    def search(self, query: str, k: int = 3):
        """요약된 내용으로 검색"""
        results = self.vectorstore.similarity_search(query, k=k)
        return results

In [9]:
# 사용 예시
science_rag = ScienceRAG()

science_rag.add_documents(df)



In [16]:
query = "오토마톤의 특징에 대해 알려줘."
docs = science_rag.vectorstore.similarity_search_with_score(query, k=3)
docs

[(Document(id='70d104b2-8d74-4799-a09d-5a4c8dd577c0', metadata={'docid': '70d104b2-8d74-4799-a09d-5a4c8dd577c0', 'content': '언어 {ww | w in (0 + 1)*}는 특정한 조건을 만족하는 문자열의 집합입니다. 이 언어는 일부 튜링 기계에 의해 허용되지만, 푸시다운 오토마톤에 의해서는 허용되지 않습니다. 튜링 기계는 다양한 계산 작업을 수행할 수 있는 추상적인 컴퓨팅 장치입니다. 그러나 푸시다운 오토마톤은 스택을 사용하여 제한된 형태의 계산 작업만을 수행할 수 있습니다. 따라서 언어 {ww | w in (0 + 1)*}는 튜링 기계에 의해 허용되지만, 푸시다운 오토마톤에 의해서는 허용되지 않습니다. 이러한 언어의 특성은 컴퓨터 과학 분야에서 중요한 개념으로 다루어지고 있습니다.'}, page_content='언어 {ww | w in (0 + 1)*}는 튜링 기계에 의해 허용되지만 푸시다운 오토마톤에 의해 허용되지 않으며, 이는 스택 기반 계산의 제한성과 튜링 기계의 계산 능력 차이를 보여주는 핵심 사례이다.'),
  np.float32(0.9753531)),
 (Document(id='651c1b65-b620-4abf-bfcd-7e6753e64142', metadata={'docid': '651c1b65-b620-4abf-bfcd-7e6753e64142', 'content': '지구는 자전축을 중심으로 하루에 한 번 자전합니다. 이는 지구의 자전 속도와 자전축의 기울기에 의해 결정됩니다. 자전축은 지구의 지평선과 약 23.5도의 각도로 기울어져 있으며, 이로 인해 지구는 하루에 한 번 자전합니다. 이 자전은 지구의 낮과 밤의 변화를 만들어내고, 시간의 흐름을 결정하는 중요한 역할을 합니다. 따라서, 지구는 매일 한 번 자전함으로써 우리에게 일상 생활에서 익숙한 낮과 밤의 변화를 제공합니다.'}, page_content='지구는 자전축을 중심으로 자전속도와 기울

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

llm = ChatOllama(model="alibayram/Qwen3-30B-A3B-Instruct-2507")

# 프롬프트 
convertFormat = """
당신은 문자열 포맷 마이그레이션 전문가 입니다. <message> 에서 content 내용만 문자열로 출력합니다. 만일 content가 여러개가 있으면 전체적인 문맥을 파악하여 질문을 만들어서 출력합니다.

<message>
{message}
</message>

[example]

    <message>{"role": "user", "content": "피를 맑게 하고 몸 속의 노폐물을 없애는 역할을 하는 기관은?"}</message> 
    output:피를 맑게 하고 몸 속의 노폐물을 없애는 역할을 하는 기관은? 

    <message>{"role": "user", "content": "이란 콘트라 사건이 뭐야"}, {"role": "assistant", "content": "이란-콘트라 사건은 로널드 레이건 집권기인 1986년에 레이건 행정부와 CIA가 적성국이었던 이란에게 무기를 몰래 수출한 대금으로 니카라과의 우익 성향 반군 콘트라를 지원하면서 동시에 반군으로부터 마약을 사들인 후 미국에 판매하다가 발각되어 큰 파장을 일으킨 사건입니다."}, {"role": "user", "content": "이 사건이 미국 정치에 미친 영향은?"}</message>
    output:1986년에 발생한 이란 콘트라 사건이 미국 정치에 미친 영향은? 


output:
[Your output here - NOTHING ELSE]
"""

# 프롬프트 객체 생성
convertFormat_prompt = PromptTemplate(
    input_variables=["message"],
    template=convertFormat
)

# 출력 파서 (문자열)
output_parser = StrOutputParser()

# LCEL 체인 구성
convertFormat_chain = (
    convertFormat_prompt 
    | llm 
    | output_parser
)

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama


# 프롬프트 
selectYn = """
당신은 과학 상식 전문가 입니다. 만약 <message> 가 과학 상식과 관련된 질문이라면  Y 아니라면 N으로 답해주세요.

<message>
{message}
</message>

당신은 반드시 Y 뜨는 N으로 답해야 합니다.
Answer:
"""

# 프롬프트 객체 생성
selectYn_prompt = PromptTemplate(
    input_variables=["message"],
    template=selectYn
)

# 출력 파서 (문자열)
output_parser = StrOutputParser()

# LCEL 체인 구성
selectYn_chain = (
    selectYn_prompt 
    | llm 
    | output_parser
)

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama


# 프롬프트 
answer = """
당신은 과학 상식 전문가 입니다. <message> 의 질문에 대해서 주어진 <reference> 정보를 활용하여 간결하게 답변을 생성합니다.

    - 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답합니다.
    - 한국어로 답변을 생성합니다..

<message>
{message}
</message>

<reference>
{reference}
</reference>

Answer:
"""

# 프롬프트 객체 생성
answer_prompt = PromptTemplate(
    input_variables=["message"],
    template=answer
)

# 출력 파서 (문자열)
output_parser = StrOutputParser()

# LCEL 체인 구성
answer_chain = (
    answer_prompt 
    | llm 
    | output_parser
)

In [ ]:
def query_db(message):

    docs = science_rag.vectorstore.similarity_search_with_relevance_scores(query, k=3)
    filtered_docs = [(doc, score) for doc, score in docs if score <= 0.8] 

    return 

In [ ]:
import json

# LLM과 검색엔진을 활용한 RAG 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "standalone_query", "topk": [], "references": [], "answer": ""}

    message = convertFormat_chain.invoke({"message": messages})
    result = selectYn_chain.invoke({"document": message})

    context = {"message":message}


    if result == "Y":
        context["reference"], response["topk"], response["references"] = query_db(message)
    else:
        context["reference"] = ""
        response["topk"] = []
        response["references"] = []
    
    response["answer"] = answer_chain.invoke(context)



    """ if result.choices[0].message.tool_calls:
        tool_call = result.choices[0].message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        standalone_query = function_args.get("standalone_query")

        # Baseline으로는 sparse_retrieve만 사용하여 검색 결과 추출
        search_result = sparse_retrieve(standalone_query, 3)

        response["standalone_query"] = standalone_query
        retrieved_context = []
        for i,rst in enumerate(search_result['hits']['hits']):
            retrieved_context.append(rst["_source"]["content"])
            response["topk"].append(rst["_source"]["docid"])
            response["references"].append({"score": rst["_score"], "content": rst["_source"]["content"]})

        content = json.dumps(retrieved_context)
        messages.append({"role": "assistant", "content": content})
        msg = [{"role": "system", "content": persona_qa}] + messages
        try:
            qaresult = client.chat.completions.create(
                    model=llm_model,
                    messages=msg,
                    temperature=0,
                    seed=1,
                    timeout=30
                )
        except Exception as e:
            traceback.print_exc()
            return response
        response["answer"] = qaresult.choices[0].message.content

    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content """

    return response


def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1


In [ ]:
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("./data/eval.jsonl", "sample_submission.csv")